In [1]:
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [23]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFile, CogniteFileApply

In [3]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [4]:
SPACE = "sp_upgrade"

In [5]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Happy-Path

In [6]:
file = FileMetadataWrite(
    external_id="file_case1_happy_path",
    name="File Case 1 Happy Path",
    mime_type="text/plain",
)

In [7]:
# Ensure the file does not exist before running the test
client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)

In [8]:
created, upload_link = client.files.create(file)
created

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
labels,[]
id,1956983605665849
created_time,2025-07-10 05:29:57.755000
last_updated_time,2025-07-10 05:29:57.755000
uploaded,False


In [9]:
filecontent = b"This is filecontent"

In [10]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,1956983605665849
created_time,2025-07-10 05:29:57.755000
last_updated_time,2025-07-10 05:29:57.755000
uploaded,False


In [11]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    updated

In [12]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name="Case 1 Happy Path",
)

In [13]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,file_case1_happy_path,1,True,2025-07-10 05:30:11.516,2025-07-10 05:30:11.516


In [14]:
retrieved_content = client.files.download_bytes(instance_id=cognite_file.as_id())

In [15]:
retrieved_content

b'This is filecontent'

In [16]:
newcontent = b"This is new filecontent"

In [17]:
client.files.upload_content_bytes(newcontent, instance_id=cognite_file.as_id())

,value
external_id,file_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."
name,File Case 1 Happy Path
mime_type,text/plain
id,1956983605665849
created_time,2025-07-10 05:29:57.755000
last_updated_time,2025-07-10 05:30:04.731000
uploaded,True
uploaded_time,2025-07-10 05:30:01.385000


In [18]:
retrieved_updatedcontent = client.files.download_bytes(external_id=file.external_id)

In [19]:
retrieved_updatedcontent

b'This is new filecontent'

In [20]:
retrieved_file = client.files.retrieve(external_id=file.external_id)
retrieved_file

,value
external_id,file_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."
name,File Case 1 Happy Path
mime_type,text/plain
id,1956983605665849
created_time,2025-07-10 05:29:57.755000
last_updated_time,2025-07-10 05:30:04.731000
uploaded,True
uploaded_time,2025-07-10 05:30:19.579000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."


In [24]:
retrieved_cognite_file = client.data_modeling.instances.retrieve_nodes(cognite_file.as_id(), node_cls=CogniteFile)

In [25]:
retrieved_cognite_file.dump()

{'space': 'sp_upgrade',
 'externalId': 'file_case1_happy_path',
 'version': 2,
 'lastUpdatedTime': 1752125420023,
 'createdTime': 1752125411516,
 'instanceType': 'node',
 'properties': {'cdf_cdm': {'CogniteFile/v1': {'name': 'Case 1 Happy Path',
    'description': None,
    'tags': None,
    'aliases': None,
    'sourceId': None,
    'sourceContext': None,
    'source': None,
    'sourceCreatedTime': None,
    'sourceUpdatedTime': None,
    'sourceCreatedUser': None,
    'sourceUpdatedUser': None,
    'assets': None,
    'mimeType': None,
    'directory': None,
    'isUploaded': True,
    'uploadedTime': '2025-07-10T05:30:19.579+00:00',
    'category': None}}}}